In [61]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers import ParentDocumentRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain.storage import InMemoryStore
from dotenv import load_dotenv
import os

In [47]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [48]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

llm = ChatOpenAI(model_name='gpt-3.5-turbo', max_tokens=500)

In [49]:
# Carregar o PDF

pdf_link = 'pdf-investimentos-RAG.pdf'
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [50]:
len(pages)

25

In [51]:
# Splitter 
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [52]:
# Storages
store = InMemoryStore()
vectorstore = Chroma(embedding_function=embeddings, persist_directory='childVectorDB')

In [53]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)


In [ ]:
parent_document_retriever.vectorstore.get()

In [62]:
TEMPLATE =  """"
    Você é um especialista em investimentos e temas do tal. Responda a perguntas utilizando o contexto informado.
    Query: 
    {question}
    
    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [63]:
setup_retrival = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": parent_document_retriever
})

output_parser = StrOutputParser()

In [64]:
parent_chain_retrival = setup_retrival | rag_prompt | llm | output_parser

In [68]:
parent_chain_retrival.invoke("Qual a diferença entre renda fixa e renda variável?")

'Na renda fixa, a remuneração ou sua forma de cálculo é previamente definida no momento da aplicação. O investidor sabe exatamente o valor que receberá no vencimento, sendo chamado de renda fixa pré-fixado. Já na renda variável, o investidor não sabe antecipadamente qual será a rentabilidade da aplicação, sendo variável e dependente de inúmeros fatores. Em resumo, na renda fixa o retorno é previsível, enquanto na renda variável o retorno é incerto e dependente do desempenho futuro da empresa.'